In [1]:
import shutil

# importing the sys module
import sys        

# appending the directory of mod.py
# in the sys.path list
sys.path.append('../')   

import json
import os
import random
from collections import defaultdict

import torch
from PIL import Image
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.optim import lr_scheduler
from torch.optim import Adam
import numpy as np
from sklearn.model_selection import train_test_split

from datetime import datetime


import compare_pipeline__config as cfg

from Common.Datasets.Morph2.data_parser import DataParser
from Common.Datasets.Morph2.Morph2ClassifierDataset2 import Morph2ClassifierDataset2
from Common.Datasets.Morph2.Morph2ClassifierDataset3 import Morph2ClassifierDataset3
#from diff_pipeline4__train import train_diff_cls_model_iter

from Common.Optimizers.RangerLars import RangerLars
from Common.Schedulers.GradualWarmupScheduler import GradualWarmupScheduler


from compare_pipeline__model import ComparePipelineModel, ComparePipelineDeepModel, ComparePipelineDeepModel2, DiffModelConfigType


from Common.Pipelines.AgePredictSingleRefPipeline import AgePredictSearchPipeline, AgePredictSingleRefComparePipeline, AgePredictSingleRefPipeline
#from Common.Analysis.DiffAnalysisMethods import diff_pipeline_confusion_matrix_analysis
from Common.Analysis.DiffAnalysisMethods import evaluate_pipeline_clean_compare_enhanced


from tqdm import tqdm


from Common.Models.UnifiedClassificaionAndRegressionAgeModel import UnifiedClassificaionAndRegressionAgeModel
from Common.Models.transformer import *
from Common.Models.unified_transformer_model import AgeTransformer

from sklearn import utils


if __name__ == "__main__":

    torch.manual_seed(cfg.RANDOM_SEED)
    np.random.seed(cfg.RANDOM_SEED)
    random.seed(cfg.RANDOM_SEED)

    if cfg.USE_GPU:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device("cpu")

    print(device)

    torch.cuda.empty_cache()





    # Load data
    data_parser = DataParser('../Common/Datasets/Morph2/aligned_data/aligned_dataset_with_metadata_uint8.hdf5', small_data=cfg.SMALL_DATA)
    data_parser.initialize_data()

    x_train, y_train, x_test, y_test = data_parser.x_train,	data_parser.y_train, data_parser.x_test, data_parser.y_test,
    if cfg.RANDOM_SPLIT:
        all_images = np.concatenate((x_train, x_test), axis=0)
        all_labels = np.concatenate((y_train, y_test), axis=0)

        x_train, x_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=cfg.TEST_SIZE_FOR_RS, random_state=cfg.RANDOM_SEED)

    face2emb_arr_trn_r = np.load('face2emb_arr_trn.npy', allow_pickle=True)
#     import pdb
#     pdb.set_trace()
    #face2emb_arr_trn_r = face2emb_arr_trn_r.item()
    
    face2emb_arr_vld_r = np.load('face2emb_arr_vld.npy', allow_pickle=True)
    #face2emb_arr_vld_r = face2emb_arr_vld_r.item()
    
    # shuffle basic aligned test
    # x_test_shuffled
    # y_test_shuffled
    

    with open("im2age_map_test.json", 'r') as im2age_map_test_f:
        im2age_map_test = json.load(im2age_map_test_f)

#     transf_age_pred_model = transforms.Compose([
#             transforms.RandomResizedCrop(224, (0.9, 1.0)),
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomApply([transforms.ColorJitter(
#                 brightness=0.1,
#                 contrast=0.1,
#                 saturation=0.1,
#                 hue=0.1
#             )], p=0.5),
#             transforms.RandomApply([transforms.RandomAffine(
#                 degrees=10,
#                 translate=(0.1, 0.1),
#                 scale=(0.9, 1.1),
#                 shear=5,
#                 resample=Image.BICUBIC
#             )], p=0.5),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#             transforms.RandomErasing(p=0.5)
#         ])



#################################
# transformsa with less random
###################
#     transf_diff_model = transforms.Compose([
#             transforms.Resize(224), # just for testing
#     #          transforms.RandomResizedCrop(224, (0.9, 1.0)),
#     #          transforms.RandomHorizontalFlip(),
#             transforms.RandomApply([transforms.ColorJitter(
#                 brightness=0.1,
#                 contrast=0.1,
#                 saturation=0.1,
#                 hue=0.1
#             )], p=0.5),
#     #         transforms.RandomApply([transforms.RandomAffine(
#     #             degrees=10,
#     #             translate=(0.1, 0.1),
#     #             scale=(0.9, 1.1),
#     #             shear=5,
#     #             resample=Image.BICUBIC
#     #         )], p=0.5),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     #        transforms.RandomErasing(p=0.5)
#     ])


    transf_diff_model = transforms.Compose([
            transforms.RandomResizedCrop(224, (0.9, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.ColorJitter(
                brightness=0.1,
                contrast=0.1,
                saturation=0.1,
                hue=0.1
            )], p=0.5),
            transforms.RandomApply([transforms.RandomAffine(
                degrees=10,
                translate=(0.1, 0.1),
                scale=(0.9, 1.1),
                shear=5,
                resample=Image.BICUBIC
            )], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            transforms.RandomErasing(p=0.5)
        ])


    test_ds = Morph2ClassifierDataset3( #Morph2ClassifierDataset Morph2ClassifierDataset2
        x_test,
        y_test,
        cfg.MIN_AGE,
        cfg.AGE_INTERVAL,
        transform=transforms.Compose([
            transforms.RandomResizedCrop(224, (0.9, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.ColorJitter(
                brightness=0.1,
                contrast=0.1,
                saturation=0.1,
                hue=0.1
            )], p=0.5),
            transforms.RandomApply([transforms.RandomAffine(
                degrees=10,
                translate=(0.1, 0.1),
                scale=(0.9, 1.1),
                shear=5,
                resample=Image.BICUBIC
            )], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            transforms.RandomErasing(p=0.5)
        ]),
        copies=cfg.NUM_OF_COPIES_AGE_PREDICT
    )

    image_datasets = {
        'train': [],
        'val': test_ds
    }

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

    data_loaders = {
        'train': None, #DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=True, drop_last=True),
        'val': DataLoader(test_ds, batch_size=1, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=False)
    }


    num_classes = int((cfg.MAX_AGE - cfg.MIN_AGE) / cfg.AGE_INTERVAL + 1)


    # create model and parameters
    if cfg.DEEP_MODEL:
        # model = ComparePipelineDeepModel(age_interval=cfg.AGE_INTERVAL, 
        #                         min_age=cfg.MIN_AGE, 
        #                         max_age=cfg.MAX_AGE, 
        #                         age_radius=cfg.AGE_RADIUS, 
        #                         device=device)#,
                                #num_references=cfg.NUM_OF_REFS)

        model = ComparePipelineDeepModel2(age_interval=cfg.AGE_INTERVAL, 
                                min_age=cfg.MIN_AGE, 
                                max_age=cfg.MAX_AGE, 
                                age_radius=cfg.AGE_RADIUS, 
                                device=device, 
                                deep=True, 
                                num_references=1,#cfg.NUM_OF_REFERENCES, 
                                config_type=DiffModelConfigType.AddedEmbeddingAndMlpHeadWithDiffHead, 
                                added_embed_layer_size=512, 
                                diff_embed_layer_size=512,
                                is_ordinal_reg=True)
    else:
        model = ComparePipelineModel(age_interval=cfg.AGE_INTERVAL, 
                                min_age=cfg.MIN_AGE, 
                                max_age=cfg.MAX_AGE, 
                                age_radius=cfg.AGE_RADIUS, 
                                device=device)

    print("testing set size: " + str(len(test_ds)))
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_30_10_2022_17_23_12' # trained in range 0-3 with close embs (from 25 percentile), unfreeze backbone in epoch 5
    #model_filename = 'weights_11064_1.3333.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_04_11_2022_04_00_05' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_6915_1.3252.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_18_11_2022_11_02_25' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_5532_1.3246.pt'

    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_04_11_2022_21_42_24' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 10
    #model_filename = 'weights_16596_1.3250.pt'

    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_06_11_2022_11_38_47' # trained in range 0-4 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_11064_1.6606.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_08_11_2022_01_46_52' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_4200_1.3240.pt'
    
    #model_dir = 'D:/projects/age_estimation/DiffPipeline4/weights/Morph2Diff/unified/iter/time_10_11_2022_09_20_42' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    #model_filename = 'weights_2766_1.3247.pt'
    
    model_dir = 'D:/projects/age_estimation/ComparePipeline/weights/Morph2Diff/unified/iter/time_25_11_2022_16_18_25' # trained in range 0-3 with close embs (from 5 closest), unfreeze backbone in epoch 5
    model_filename = 'weights_31809_0.8688.pt'


    
    pipeline = AgePredictSingleRefComparePipeline(#AgePredictSearchPipeline(
                        device,
                        num_classes, 
                        cfg.AGE_INTERVAL, 
                        cfg.MIN_AGE, 
                        cfg.MAX_AGE, 
                        cfg.AGE_RADIUS,
                        cfg.MID_FEATURE_SIZE_AGE_PREDICT, 
                        x_train,
                        y_train,
                        0, #cfg.DEEP, 
                        0, #cfg.NUM_OF_REFERENCES, 
                        0, #cfg.DIFF_MODEL_CONFIG_TYPE, 
                        0, #cfg.ADDED_EMBED_LAYER_SIZE,
                        0, #cfg.DIFF_EMBED_LAYER_SIZE,
                        transf_diff_model,
                        cfg.NUM_OF_COPIES_AGE_PREDICT,
                        UnifiedClassificaionAndRegressionAgeModel,
                        TransformerModel,
                        AgeTransformer,
                        model, #AgeDiffModelNonOpt, #AgeDiffModel,
                        True, #cfg.IS_ORDINAL_REG,
                        no_age_transformer_init=False,
                        cfg=cfg,
                        age_diff_model_path_arg=model_dir,
                        age_diff_model_file_name_arg=model_filename,
                        embs_trn=face2emb_arr_trn_r,
                        embs_vld=face2emb_arr_vld_r)


    #diff_pipeline_confusion_matrix_analysis(pipeline, data_loaders['val'], device, dataset_sizes['val'], is_ordinal_reg=True)


    if cfg.CHECK_DIFF_BASED:
        mae_on_test = evaluate_pipeline_clean_compare_enhanced(pipeline, 
                                                        data_loaders['val'], 
                                                        device, 
                                                        dataset_sizes['val'], 
                                                        bypass_diff=False, 
                                                        compare_post_round=False, 
                                                        range_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, 
                                                        range_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI, 
                                                        im2age_map_test=im2age_map_test, 
                                                        check_against_pred_db=False)
    else:
        mae_on_test = evaluate_pipeline_clean_compare_enhanced(pipeline, 
                                                    data_loaders['val'], 
                                                    device, 
                                                    dataset_sizes['val'], 
                                                    bypass_diff=True, 
                                                    compare_post_round=False, 
                                                    range_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, 
                                                    range_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI, 
                                                    im2age_map_test=im2age_map_test, 
                                                    check_im2age_map_test=False,
                                                    create_im2age_map_test=False, 
                                                    check_against_pred_db=False)
        #mae_on_test = evaluate_pipeline_clean_enhanced(pipeline, data_loaders['val'], device, dataset_sizes['val'], bypass_diff=True, compare_post_round=False, range_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, range_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI, im2age_map_test=im2age_map_test)



cpu


C:\Users\proj\.conda\envs\my_thesis_py38\lib\site-packages\torchvision\transforms\transforms.py:1361: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


testing set size: 10606
running on validation set...


D:\projects\age_estimation\ComparePipeline\..\Common\Models\transformer.py:127: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = nn.Softmax()(classification_logits)


real_age: tensor([67.]), age_est_pre: 58.53836441040039, age_est_post : 59.53836441040039, ref_age : 67, good fix !
real_age: tensor([59.]), age_est_pre: 52.78510284423828, age_est_post : 53.78510284423828, ref_age : 58, good fix !
real_age: tensor([60.]), age_est_pre: 57.26027297973633, age_est_post : 58.26027297973633, ref_age : 69, good fix !
real_age: tensor([60.]), age_est_pre: 56.67746353149414, age_est_post : 57.67746353149414, ref_age : 64, good fix !
real_age: tensor([60.]), age_est_pre: 57.131263732910156, age_est_post : 58.131263732910156, ref_age : 63, good fix !
real_age: tensor([58.]), age_est_pre: 51.00600051879883, age_est_post : 52.00600051879883, ref_age : 57, good fix !
real_age: tensor([61.]), age_est_pre: 55.07495880126953, age_est_post : 56.07495880126953, ref_age : 61, good fix !
real_age: tensor([64.]), age_est_pre: 58.74800491333008, age_est_post : 59.74800491333008, ref_age : 70, good fix !
real_age: tensor([64.]), age_est_pre: 60.12752914428711, age_est_post 

...

real_age: tensor([19.]), age_est_pre: 20.014957427978516, age_est_post : 19.014957427978516, ref_age : 18, good fix !
real_age: tensor([42.]), age_est_pre: 41.97597122192383, age_est_post : 42.97597122192383, ref_age : 47, bad fix !
real_age: tensor([28.]), age_est_pre: 31.23493766784668, age_est_post : 30.23493766784668, ref_age : 22, good fix !
real_age: tensor([22.]), age_est_pre: 20.88619041442871, age_est_post : 19.88619041442871, ref_age : 19, bad fix !
real_age: tensor([23.]), age_est_pre: 20.164104461669922, age_est_post : 19.164104461669922, ref_age : 17, bad fix !
real_age: tensor([19.]), age_est_pre: 20.20144271850586, age_est_post : 19.20144271850586, ref_age : 18, good fix !
real_age: tensor([20.]), age_est_pre: 21.561126708984375, age_est_post : 20.561126708984375, ref_age : 16, good fix !
real_age: tensor([46.]), age_est_pre: 45.80423355102539, age_est_post : 46.80423355102539, ref_age : 52, bad fix !
real_age: tensor([17.]), age_est_pre: 19.03630256652832, age_est_post 

real_age: tensor([21.]), age_est_pre: 19.00910758972168, age_est_post : 18.00910758972168, ref_age : 19, bad fix !
real_age: tensor([22.]), age_est_pre: 21.319122314453125, age_est_post : 20.319122314453125, ref_age : 16, bad fix !
real_age: tensor([19.]), age_est_pre: 20.040374755859375, age_est_post : 19.040374755859375, ref_age : 19, good fix !
real_age: tensor([18.]), age_est_pre: 17.19247055053711, age_est_post : 16.19247055053711, ref_age : 17, bad fix !
real_age: tensor([21.]), age_est_pre: 19.03074836730957, age_est_post : 18.03074836730957, ref_age : 18, bad fix !
real_age: tensor([18.]), age_est_pre: 18.373926162719727, age_est_post : 17.373926162719727, ref_age : 16, bad fix !
- IN RANGE:  6550
- IN RANGE mae sum :  tensor(9167.5508)
- OUT OF RANGE:  3401
- OUT OF RANGE mae sum :  tensor(17073.1504)
- total:  9951
- total mae sum :  tensor(26240.6582)
progress : 93.82%
current MAE (in range): 1.3996
current MAE (out of range): 5.0200
current MAE (total): 2.6370
current MAE d

real_age: tensor([16.]), age_est_pre: 16.130081176757812, age_est_post : 15.130081176757812, ref_age : 16, bad fix !
real_age: tensor([17.]), age_est_pre: 17.04973602294922, age_est_post : 16.04973602294922, ref_age : 17, bad fix !
real_age: tensor([16.]), age_est_pre: 16.106889724731445, age_est_post : 15.106889724731445, ref_age : 16, bad fix !
real_age: tensor([17.]), age_est_pre: 16.993680953979492, age_est_post : 15.993680953979492, ref_age : 16, bad fix !
real_age: tensor([17.]), age_est_pre: 17.50877571105957, age_est_post : 16.50877571105957, ref_age : 17, good fix !
real_age: tensor([21.]), age_est_pre: 19.00982093811035, age_est_post : 18.00982093811035, ref_age : 18, bad fix !
real_age: tensor([17.]), age_est_pre: 17.847259521484375, age_est_post : 16.847259521484375, ref_age : 16, good fix !
- IN RANGE:  6691
- IN RANGE mae sum :  tensor(9339.9824)
- OUT OF RANGE:  3460
- OUT OF RANGE mae sum :  tensor(17373.0176)
- total:  10151
- total mae sum :  tensor(26712.9473)
progre

real_age: tensor([19.]), age_est_pre: 16.961076736450195, age_est_post : 15.961076736450195, ref_age : 16, bad fix !
real_age: tensor([19.]), age_est_pre: 18.313385009765625, age_est_post : 17.313385009765625, ref_age : 16, bad fix !
real_age: tensor([20.]), age_est_pre: 19.025358200073242, age_est_post : 18.025358200073242, ref_age : 18, bad fix !
real_age: tensor([16.]), age_est_pre: 20.865381240844727, age_est_post : 19.865381240844727, ref_age : 20, good fix !
real_age: tensor([16.]), age_est_pre: 19.082582473754883, age_est_post : 18.082582473754883, ref_age : 17, good fix !
real_age: tensor([16.]), age_est_pre: 18.987041473388672, age_est_post : 17.987041473388672, ref_age : 16, good fix !
real_age: tensor([17.]), age_est_pre: 19.006752014160156, age_est_post : 18.006752014160156, ref_age : 16, good fix !
- IN RANGE:  6826
- IN RANGE mae sum :  tensor(9503.9795)
- OUT OF RANGE:  3525
- OUT OF RANGE mae sum :  tensor(17699.0625)
- total:  10351
- total mae sum :  tensor(27202.9844

real_age: tensor([16.]), age_est_pre: 19.65683937072754, age_est_post : 18.65683937072754, ref_age : 18, good fix !
real_age: tensor([52.]), age_est_pre: 55.2987174987793, age_est_post : 56.2987174987793, ref_age : 61, bad fix !
real_age: tensor([52.]), age_est_pre: 52.249263763427734, age_est_post : 53.249263763427734, ref_age : 69, bad fix !
real_age: tensor([24.]), age_est_pre: 20.185760498046875, age_est_post : 19.185760498046875, ref_age : 20, bad fix !
real_age: tensor([23.]), age_est_pre: 17.931476593017578, age_est_post : 16.931476593017578, ref_age : 16, bad fix !
real_age: tensor([21.]), age_est_pre: 23.96303939819336, age_est_post : 22.96303939819336, ref_age : 18, good fix !
real_age: tensor([47.]), age_est_pre: 51.096946716308594, age_est_post : 52.096946716308594, ref_age : 62, bad fix !
real_age: tensor([47.]), age_est_pre: 50.270484924316406, age_est_post : 51.270484924316406, ref_age : 57, bad fix !
real_age: tensor([42.]), age_est_pre: 44.95560073852539, age_est_post 